In [45]:
import configparser
config = configparser.RawConfigParser()
config.read("../api_key.ini")

import cohere
API_KEY = config[config.sections()[0]]['api_key']
co = cohere.Client(API_KEY)

### Load Data

In [46]:
import pandas as pd
# Load data to pandas dataframes:
sales_by_day = pd.read_csv('../data/raw/LastWeekSalesByDay.csv', delimiter=';')
top_customers = pd.read_csv('../data/raw/LastWeekTopCustomers.csv', delimiter=';')
top_products = pd.read_csv('../data/raw/LastWeekTopProducts.csv', delimiter=';')
order_statistics = pd.read_csv('../data/raw/LastWeekOrderStatistics.csv', delimiter=';')
cash_flow_report = pd.read_csv('../data/raw/CashFlowReport.csv', delimiter=';')
understocked_products = pd.read_csv('../data/raw/UnderstockedProducts.csv', delimiter=';')

### Products

In [61]:
import csv

# Initialize an empty dictionary to store the data
product_revenue = {}

# Read the CSV file and populate the dictionary
with open('../data/raw/LastWeekTopProducts.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';')
    for row in reader:
        product_description = row['Product Description']
        revenue = int(row['Revenue'])
        product_revenue[product_description] = revenue

# Print the dictionary to verify
print(product_revenue)


Customers

In [ ]:
import csv

# Initialize an empty dictionary to store the data
customer_revenue = {}

# Read the CSV file and populate the dictionary
with open('../data/raw/LastWeekTopCustomers.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';')
    for row in reader:
        customer_name = row['Customer Name']
        revenue = int(row['Revenue'])
        customer_revenue[customer_name] = revenue

# Print the dictionary to verify
print(customer_revenue)


### Sales By Day

In [ ]:
import csv

# Initialize an empty dictionary to store the data
revenue_data = {}

# Read the CSV file and populate the dictionary
with open('../data/raw/LastWeekSalesByDay.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';')
    for row in reader:
        date = row['Date']
        product_category = row['Product category']
        revenue = int(row['Revenue'])
        
        # If the product category already exists in the dictionary, update its revenue
        if product_category in revenue_data:
            revenue_data[product_category][date] = revenue
        # If the product category is new, add it to the dictionary with its revenue for the current date
        else:
            revenue_data[product_category] = {date: revenue}

# Print the dictionary to verify
print(revenue_data)


### Order Statistics

In [ ]:
import csv

# Initialize an empty dictionary to store the data
channel_orders = {}

# Read the CSV file and populate the dictionary
with open('../data/raw/LastWeekOrderStatistics.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';')
    for row in reader:
        channel = row['Channel']
        orders_count = int(row['Orders Count'])
        order_lines = int(row['Order Lines'])
        quantity_pieces = int(row['Quantity in pieces'])
        quantity_boxes = int(row['Quantity in boxes'])
        amount = int(row['Amount'])
        fulfillment_percentage = int(row['Percentage of fulfillment'])
        
        # Store the data in the dictionary
        channel_orders[channel] = {
            'Orders Count': orders_count,
            'Order Lines': order_lines,
            'Quantity in pieces': quantity_pieces,
            'Quantity in boxes': quantity_boxes,
            'Amount': amount,
            'Percentage of fulfillment': fulfillment_percentage
        }

# Print the dictionary to verify
print(channel_orders)


### Cashflow report

In [ ]:
import csv

# Initialize an empty dictionary to store the data
financial_data = {}

# Read the CSV file and populate the dictionary
with open('../data/raw/CashFlowReport.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';')
    for row in reader:
        debit = int(row['Debit'])
        credit = int(row['Credit'])
        balance = int(row['Balance'])
        inflows_forecast = int(row['Inflows Forecast'])
        outflows_forecast = int(row['Outflows Forecast'])
        projected_balance = int(row['Projected Balance'])
        
        # Store the data in the dictionary
        financial_data['Debit'] = debit
        financial_data['Credit'] = credit
        financial_data['Balance'] = balance
        financial_data['Inflows Forecast'] = inflows_forecast
        financial_data['Outflows Forecast'] = outflows_forecast
        financial_data['Projected Balance'] = projected_balance

# Print the dictionary to verify
print(financial_data)

### Understocked Products

In [ ]:
import csv

# Initialize an empty dictionary to store the data
understocked_product_data = {}

# Read the CSV file and populate the dictionary
with open('../data/raw/UnderstockedProducts.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';')
    for row in reader:
        product_description = row['Product Description']
        minimum_stock = int(row['Minimum stock'])
        current_stock = int(row['Current stock'])
        expected_qty = int(row['Expected Qty'])
        required_stock = int(row['Required stock'])
        
        # Store the data in the dictionary
        understocked_product_data[product_description] = {
            'Minimum stock': minimum_stock,
            'Current stock': current_stock,
            'Expected Qty': expected_qty,
            'Required stock': required_stock
        }

# Print the dictionary to verify
print(understocked_product_data)

# Write the dictionary to a new txt file
with open('../data/processed/dicts/understocked_products.txt', 'w') as txtfile:
    for product_description, product_data in understocked_product_data.items():
        txtfile.write(f'{product_description}\n')
        for key, value in product_data.items():
            txtfile.write(f'{key}: {value}\n')
        txtfile.write('\n')


### Functions

### Function Map

In [62]:
functions_map = {"calculate_total_product_revenue": calculate_total_product_revenue}

### Tools

In [63]:
tools = [
{
"name": "calculate_total_product_revenue",
"description": "Calculates the total revenue generated by all products from last week.",
"parameter_definitions": {
    "revenue": {
    "description": "Calculates the total revenue generated by all products.",
    "type": "str",
    "required": True
    } }
}]

### Intsructions

In [64]:
preamble = """
##Task & Context: You are an AI assistant integrated with the company's Enterprise Resource Planning (ERP) system containing data from the last week only.
Your role is to help employees access and understand data from the ERP system through natural language interactions.
Employees will ask you questions or make requests related to various business operations like inventory, sales, accounting, manufacturing, etc. 
Your task is to query the relevant data from the ERP databases, analyze and synthesize it as needed, and provide helpful responses to the employees.

##Style Guide:
Use clear, professional language tailored for a workplace context.
Respond succinctly when possible, but provide detailed explanations when the query requires it.
Maintain objectivity and avoid injecting personal opinions unless explicitly asked.
Speak in the first-person from the perspective of an AI assistant (e.g. "I retrieved the latest inventory data from the system.")
Use proper formatting like bulleted lists, tables, and code snippets where appropriate to make responses easier to parse.
Do not include non-requested data, but do provide relevant additional context if it can enhance the usefulness of your response.
Be polite and constructive. If you cannot fulfill a request, explain why in a respectful manner.
"""

### User's Query

In [65]:
query = "What was our total revenue from last week ?"

### Chat

In [66]:
from cohere import ChatMessage
chat = co.chat(
    message=query, 
    model="command-r", # command, command-r, command-light
    # chat_history=[
    #     ChatMessage(
    #         role="CHATBOT", message="Hi!",
    #     ), ChatMessage(
    #         role="CHATBOT", message="How can I help you today?",
    #     ),
    # ],
    preamble=preamble,
    # prompt_truncation="AUTO",
    # temperature=0.2,
    tools=tools,
    
)

print("The model recommends doing the following tool calls:")
print("\n".join(str(toolcall) for toolcall in chat.tool_calls))

The model recommends doing the following tool calls:
name='calculate_total_product_revenue' parameters={'revenue': ''}


In [54]:
import json

tool_results = []
for toolcall in chat.tool_calls:
    print(f"=== Running tool: {toolcall.tool_name}, with parameters {toolcall.parameters}")
    output=functions_map[toolcall.tool_name](**toolcall.parameters)
    outputs=[output]
    print(f"== toolresults: {outputs}")
    tool_results.append({"call": toolcall, "outputs": outputs})

print("The tool results that will be fed back to the model are:")
print(json.dumps(tool_results, indent=4))

AttributeError: 'ToolCall' object has no attribute 'tool_name'

In [ ]:
response = co.chat(message=query, tools=tools, tool_results=tool_results, preamble=preamble, model="command-r",temperature=0.3)